# BatchFixedLagSmoother

## Overview

The `BatchFixedLagSmoother` class in GTSAM is designed for fixed-lag smoothing in nonlinear factor graphs. It maintains a sliding window of the most recent variables and marginalizes out older variables. This is particularly useful in real-time applications where memory and computational efficiency are critical.

This fixed lag smoother will **batch-optimize** at every iteration, but warm-started from the last estimate.

## Mathematical Formulation

The `BatchFixedLagSmoother` operates on the principle of fixed-lag smoothing, where the objective is to estimate the state $\mathbf{x}_t$ given all measurements up to time $t$, but only retaining a fixed window of recent states. The optimization problem can be expressed as:
$$
\min_{\mathbf{x}_{t-L:t}} \sum_{i=1}^{N} \| \mathbf{h}_i(\mathbf{x}_{t-L:t}) - \mathbf{z}_i \|^2
$$
where $L$ is the fixed lag, $\mathbf{h}_i$ are the measurement functions, and $\mathbf{z}_i$ are the measurements.
In practice, the functions $\mathbf{h}_i$ depend only on a subset of the state variables $\mathbf{X}_i$, and the optimization is performed over a set of $N$ *factors* $\phi_i$ instead:
$$
\min_{\mathbf{x}_{t-L:t}} \sum_{i=1}^{N} \| \phi_i(\mathbf{X}_i; \mathbf{z}_i) \|^2
$$
The API below allows the user to add new factors at every iteration, which will be automatically pruned after they no longer depend on any variables in the lag.

## API

### Constructor

You construct a `BatchFixedLagSmoother` object with the following parameters:

- **smootherLag**: The length of the smoother lag. Any variable older than this amount will be marginalized out. *(Default: 0.0)*
- **parameters**: The Levenberg-Marquardt optimization parameters. *(Default: `LevenbergMarquardtParams()`)*  
- **enforceConsistency**: A flag indicating whether the optimizer should enforce probabilistic consistency by maintaining the linearization point of all variables involved in linearized/marginal factors at the edge of the smoothing window. *(Default: `true`)*

### Smoothing and Optimization

- **update**: This method is the core of the `BatchFixedLagSmoother`. It processes new factors and variables, updating the current estimate of the state. The update method also manages the marginalization of variables that fall outside the fixed lag window.

### Computational Considerations

Every call to `update` triggers a batch LM optimization: use the parameters to control the convergence thresholds to bound computation to fit within your application.

## Internals

- **marginalize**: This function handles the marginalization of variables that are no longer within the fixed lag window. Marginalization is a crucial step in maintaining the size of the factor graph, ensuring that only relevant variables are kept for optimization.
